In [1]:
import fasttext
model = fasttext.load_model('./util/cc.en.300.bin')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import re

from tqdm import tqdm

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import util.normalization as norm
import util.model as models
import util.utility as util

# Import Data

In [4]:
# Opening JSON file
with open("./clothes_result/map_second_clothes.json") as json_file:
    map_second_keys = json.load(json_file)

with open("./clothes_result/map_category_clothes.json") as json_file:
    map_category = json.load(json_file)

with open("./clothes_result/list_first_cycle_clothes.json") as json_file:
    list_first_cycle = json.load(json_file)

In [5]:
# # Open residu

# df_residu = pd.read_csv("./mcd_result/residu_mcd.csv").fillna("")
# print(df_residu.info())
# df_residu['residu'] = df_residu['residu'].apply(lambda x: str(x).split(', '))

In [6]:
df_base = pd.read_csv("./clothes_result/base_clothes_v1.csv")

df_base['aspect'] = df_base['aspect'].apply(lambda x: x.split(", "))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: x.split(".\n"))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: x.split(".\n"))

df_base['residu'] = df_base['residu'].apply(lambda x: str(x).split(', '))

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewer_id     100 non-null    int64 
 1   review          100 non-null    object
 2   summarize       100 non-null    object
 3   aspect          100 non-null    object
 4   token_sentence  100 non-null    object
 5   token_lemma     100 non-null    object
 6   rating          100 non-null    int64 
 7   residu          100 non-null    object
dtypes: int64(2), object(6)
memory usage: 6.4+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu
0,0,So flattering and so comfortable! I am in love...,So flattering and so comfortable! I am in lov...,"[beige sandal, casual look, compliment, left s...","[It has a relaxed fit style as described, and ...","[it have a relaxed fit style as described, and...",5,"[more item, profile, grocery, beige sandal, cr..."
1,1,"Cute but It Pills Great color and pattern, wit...","Material began to pill after first wearing, a...","[great color, material, slouchy look, wash cyc...","[Cute but It Pills Great color and pattern, wi...","[cute but it pill great color and pattern, wit...",3,"[first washing, wash cycle, slouchy look, hand..."
2,2,It fell apart after one wash One set fell apar...,One set fell apart after one wash . Threads c...,"[bad thing, string, set, thread, product info,...",[It fell apart after one wash One set fell apa...,[it fell apart after one wash one set fell apa...,1,"[tie, product info, bad thing, string, set]"
3,3,"Comfortable, nice color, but LOTS of pilling. ...","Comfortable, nice color, but lots of pilling ...","[purchase, good care, nice color, fit, fabric,...","[nice color, I like the fit, weight, I would h...","[nice color, i like the fit, weight, i would h...",3,"[weight, thats, good care]"
4,4,"Cute dress that.even a short, skinny old woman...","Cute dress that.even a short, skinny old woma...","[necessary spot, material, old woman, woman sk...","[Cute dress that.even a short, skinny old woma...","[cute dress that.even a short, skinny old woma...",5,"[dress that.even, old woman, woman skinny, nec..."


In [7]:
df_nodes = pd.read_csv("./clothes_result/nodes_clothes.csv")
print(df_nodes.info())
df_nodes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   node_id       462 non-null    int64 
 1   reviewer_id   462 non-null    int64 
 2   first_cycle   462 non-null    object
 3   second_cycle  462 non-null    object
 4   category      462 non-null    object
dtypes: int64(2), object(3)
memory usage: 18.2+ KB
None


,node_id,reviewer_id,first_cycle,second_cycle,category
0,0,0,other,type,products
1,1,0,compliment,other,other
2,2,0,perfect,emotion,customer experience
3,3,0,length,experience,customer experience
4,4,0,pocket,properties,products


# Sentiment Analysis

## A. Sentiment Sentence

In [8]:
df_sentiment = df_base[['reviewer_id', 'token_lemma']].copy().explode('token_lemma')

tqdm.pandas()
df_sentiment['pattern_prediction'] = df_sentiment['token_lemma']\
                                    .progress_apply(models.pattern_lexicon_model)\
                                    .map({'positive': 1,'negative': -1})

df_sentiment = df_sentiment.groupby('reviewer_id')['pattern_prediction'].apply(list).reset_index()
print(df_sentiment.info())
df_sentiment

100%|██████████████████████████████████████████████████████████████████████████████| 336/336 [00:00<00:00, 5360.07it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviewer_id         100 non-null    int64 
 1   pattern_prediction  100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
None


,reviewer_id,pattern_prediction
0,0,"[1, 1, -1, 1, 1, -1, -1, 1]"
1,1,"[1, -1, 1, -1, -1]"
2,2,"[-1, -1, -1, -1]"
3,3,"[1, 1, -1, 1, 1, -1, -1]"
4,4,"[1, -1, -1]"
...,...,...
95,95,"[1, -1]"
96,96,"[-1, 1, -1]"
97,97,[1]
98,98,"[-1, -1]"


## B. Inverse Coding Process

In [9]:
# Inverse coding process

df_1 = df_base[['reviewer_id', 'aspect', 'token_lemma', 'residu']].copy()
df_2 = df_nodes[['reviewer_id', 'node_id', 'first_cycle', 'second_cycle', 'category']].copy()

df_temp = df_1.merge(df_2, on='reviewer_id', how='inner')
df_temp = df_temp.merge(df_sentiment, on='reviewer_id', how='left')
print(df_temp.info())
df_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviewer_id         462 non-null    int64 
 1   aspect              462 non-null    object
 2   token_lemma         462 non-null    object
 3   residu              462 non-null    object
 4   node_id             462 non-null    int64 
 5   first_cycle         462 non-null    object
 6   second_cycle        462 non-null    object
 7   category            462 non-null    object
 8   pattern_prediction  462 non-null    object
dtypes: int64(2), object(7)
memory usage: 32.6+ KB
None


,reviewer_id,aspect,token_lemma,residu,node_id,first_cycle,second_cycle,category,pattern_prediction
0,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",0,other,type,products,"[1, 1, -1, 1, 1, -1, -1, 1]"
1,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",1,compliment,other,other,"[1, 1, -1, 1, 1, -1, -1, 1]"
2,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",2,perfect,emotion,customer experience,"[1, 1, -1, 1, 1, -1, -1, 1]"
3,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",3,length,experience,customer experience,"[1, 1, -1, 1, 1, -1, -1, 1]"
4,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",4,pocket,properties,products,"[1, 1, -1, 1, 1, -1, -1, 1]"


In [10]:
def inverse_score(x):
    # Define storage

    keywords = [] # Keywords for pattern search
    keys = [] # Key value for storing
    scoring =  {}
    
    
    # Extract all values
    first = x['first_cycle']
    second = x['second_cycle']
    residu = x['residu']
    aspect = x['aspect']
    tokens = x['token_lemma']
    scores = x['pattern_prediction']
    category = x['category']
    
    
    
    ################### INVERSING PROCESS ##########################################
    # Get the aspect of each code
    if first == 'other':
        for res in residu:
            nearest = util.get_nearest_word(res, [second], model, threshold=.35)
            if nearest:
                keywords.append(res)
                keys.append(second)
                break
    
    else:
        pattern = re.compile(rf"{first}")
        for element in aspect:
            if pattern.search(element):
                keywords.append(element)
                keys.append(first)
                

    # If keys and keywords empty, return None
    if (len(keys) == 0) and (len(keywords) == 0):
        return
    
    ################################# SCORING ##################################################
    # Get the index sentence
    for keyword in keywords:
        pattern = re.compile(rf"{keyword}")
        
        for idx, element in enumerate(tokens):
            if pattern.search(element):
                if keyword not in scoring:
                    scoring[keyword] = scores[idx]
                else:
                    scoring[keyword] = scoring[keyword] + scores[idx]

        if keyword not in scoring:
            # If it's not detected we have to split it and choose the token that contains all element.
            temp_split_result = {}
            for e in keyword.split(' '):
                pattern = re.compile(rf"{e}")
                
                for idx, element in enumerate(tokens):
                    if pattern.search(element):
                        if keyword not in scoring:
                            scoring[keyword] = scores[idx]
                        else:
                            scoring[keyword] = scoring[keyword] + scores[idx]

    # Scoring    
    total_scores = sum(list(scoring.values()))
    tags = '/'.join(keywords)
    
    if total_scores < 0:
        return (second, tags, 'negative')
    else:
        return (second, tags, 'positive')
#     return (second, tags, total_scores)

df_temp['key_score_aspect'] = [inverse_score(x[-1]) for x in df_temp.iterrows()]
# If positive => sentiment 1, if negative => sentiment 0, if empty => sentiment 1 
df_temp['sentiment'] = df_temp['key_score_aspect'].apply(lambda x: 1 if not x else 1 if x[-1] == 'positive' else 0)

# print(inverse_score(df_temp.iloc[495]))
# print(inverse_score(df_temp.iloc[497]))

df_temp

,reviewer_id,aspect,token_lemma,residu,node_id,first_cycle,second_cycle,category,pattern_prediction,key_score_aspect,sentiment
0,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",0,other,type,products,"[1, 1, -1, 1, 1, -1, -1, 1]","(type, name, positive)",1
1,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",1,compliment,other,other,"[1, 1, -1, 1, 1, -1, -1, 1]","(other, compliment, positive)",1
2,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",2,perfect,emotion,customer experience,"[1, 1, -1, 1, 1, -1, -1, 1]","(emotion, perfect length, positive)",1
3,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",3,length,experience,customer experience,"[1, 1, -1, 1, 1, -1, -1, 1]","(experience, perfect length, positive)",1
4,0,"[beige sandal, casual look, compliment, left s...","[it have a relaxed fit style as described, and...","[more item, profile, grocery, beige sandal, cr...",4,pocket,properties,products,"[1, 1, -1, 1, 1, -1, -1, 1]","(properties, pocket, negative)",0
...,...,...,...,...,...,...,...,...,...,...,...
457,99,"[positive side, tight sleeve, thread, several ...",[with reservation first one i receive be rip a...,"[shiny band, sleeve opening, band detail, repl...",457,side,experience,customer experience,"[-1, -1, -1, 1, 1, 1]","(experience, positive side, positive)",1
458,99,"[positive side, tight sleeve, thread, several ...",[with reservation first one i receive be rip a...,"[shiny band, sleeve opening, band detail, repl...",458,piece,application,customer experience,"[-1, -1, -1, 1, 1, 1]","(application, quality piece, positive)",1
459,99,"[positive side, tight sleeve, thread, several ...",[with reservation first one i receive be rip a...,"[shiny band, sleeve opening, band detail, repl...",459,quality,properties,products,"[-1, -1, -1, 1, 1, 1]","(properties, quality piece, positive)",1
460,99,"[positive side, tight sleeve, thread, several ...",[with reservation first one i receive be rip a...,"[shiny band, sleeve opening, band detail, repl...",460,several,other,other,"[-1, -1, -1, 1, 1, 1]","(other, several place, negative)",0


# Sentiment Scoring

In [11]:
result = df_temp[['node_id', 'reviewer_id', 
                  'first_cycle', 'second_cycle', 
                  'category', 'sentiment']].copy()

result

,node_id,reviewer_id,first_cycle,second_cycle,category,sentiment
0,0,0,other,type,products,1
1,1,0,compliment,other,other,1
2,2,0,perfect,emotion,customer experience,1
3,3,0,length,experience,customer experience,1
4,4,0,pocket,properties,products,0
...,...,...,...,...,...,...
457,457,99,side,experience,customer experience,1
458,458,99,piece,application,customer experience,1
459,459,99,quality,properties,products,1
460,460,99,several,other,other,0


In [12]:
first_sentiment = result.groupby('first_cycle')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'first_cycle_sentiment_positive_rate'})

second_sentiment = result.groupby('second_cycle')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'second_cycle_sentiment_positive_rate'})

category_sentiment = result.groupby('category')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'category_sentiment_positive_rate'})


result = result.merge(first_sentiment, on='first_cycle', how='left')
result = result.merge(second_sentiment, on='second_cycle', how='left')
result = result.merge(category_sentiment, on='category', how='left')
result['overall_sentiment_positive_rate'] = round(np.sum(result['sentiment']) / result.shape[0], 3) * np.ones(result.shape[0])


result = result.drop('sentiment', axis=1)

result

,node_id,reviewer_id,first_cycle,second_cycle,category,first_cycle_sentiment_positive_rate,second_cycle_sentiment_positive_rate,category_sentiment_positive_rate,overall_sentiment_positive_rate
0,0,0,other,type,products,0.897,0.767,0.639,0.673
1,1,0,compliment,other,other,1.000,0.693,0.670,0.673
2,2,0,perfect,emotion,customer experience,1.000,0.789,0.712,0.673
3,3,0,length,experience,customer experience,1.000,0.760,0.712,0.673
4,4,0,pocket,properties,products,0.500,0.552,0.639,0.673
...,...,...,...,...,...,...,...,...,...
457,457,99,side,experience,customer experience,0.500,0.760,0.712,0.673
458,458,99,piece,application,customer experience,1.000,0.500,0.712,0.673
459,459,99,quality,properties,products,0.500,0.552,0.639,0.673
460,460,99,several,other,other,0.000,0.693,0.670,0.673


# Add Review Value

In [13]:
df_rating = df_base[['reviewer_id', 'rating']]

result = result.merge(df_rating, on='reviewer_id', how='left')

first_rating = result.groupby('first_cycle')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'first_cycle_rating'})
second_rating = result.groupby('second_cycle')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'second_cycle_rating'})
category_rating = result.groupby('category')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'category_rating'})

result = result.merge(first_rating, on='first_cycle', how='left')
result = result.merge(second_rating, on='second_cycle', how='left')
result = result.merge(category_rating, on='category', how='left')
result['overall_rating'] = round(np.sum(result['rating']) / result.shape[0], 3) * np.ones(result.shape[0])

result = result.drop('rating', axis=1)

result

,node_id,reviewer_id,first_cycle,second_cycle,category,first_cycle_sentiment_positive_rate,second_cycle_sentiment_positive_rate,category_sentiment_positive_rate,overall_sentiment_positive_rate,first_cycle_rating,second_cycle_rating,category_rating,overall_rating
0,0,0,other,type,products,0.897,0.767,0.639,0.673,3.724,4.233,3.918,3.84
1,1,0,compliment,other,other,1.000,0.693,0.670,0.673,5.000,3.816,3.750,3.84
2,2,0,perfect,emotion,customer experience,1.000,0.789,0.712,0.673,4.600,3.825,3.871,3.84
3,3,0,length,experience,customer experience,1.000,0.760,0.712,0.673,5.000,4.200,3.871,3.84
4,4,0,pocket,properties,products,0.500,0.552,0.639,0.673,3.000,3.701,3.918,3.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,457,99,side,experience,customer experience,0.500,0.760,0.712,0.673,3.167,4.200,3.871,3.84
458,458,99,piece,application,customer experience,1.000,0.500,0.712,0.673,4.250,3.438,3.871,3.84
459,459,99,quality,properties,products,0.500,0.552,0.639,0.673,2.917,3.701,3.918,3.84
460,460,99,several,other,other,0.000,0.693,0.670,0.673,3.750,3.816,3.750,3.84


In [14]:
# Update base

df_temp['key_score_aspect'] = df_temp['key_score_aspect'].apply(lambda x: ", ".join(list(x)) if x else '')
df_int = df_temp[['reviewer_id', 'key_score_aspect']]\
            .groupby('reviewer_id')\
            .agg({'key_score_aspect': lambda x: "; ".join(x)})\
            .reset_index()
df_base = df_base.merge(df_int, on='reviewer_id', how='left')

df_base['aspect'] = df_base['aspect'].apply(lambda x: ", ".join(x))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: ", ".join(x))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: ", ".join(x))
df_base['residu'] = df_base['residu'].apply(lambda x: ", ".join(x))

df_base.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,0,So flattering and so comfortable! I am in love...,So flattering and so comfortable! I am in lov...,"beige sandal, casual look, compliment, left si...","It has a relaxed fit style as described, and I...","it have a relaxed fit style as described, and ...",5,"more item, profile, grocery, beige sandal, cro...","type, name, positive; other, compliment, posit..."
1,1,"Cute but It Pills Great color and pattern, wit...","Material began to pill after first wearing, a...","great color, material, slouchy look, wash cycl...","Cute but It Pills Great color and pattern, wit...","cute but it pill great color and pattern, with...",3,"first washing, wash cycle, slouchy look, hand ...","; properties, material, positive; application,..."
2,2,It fell apart after one wash One set fell apar...,One set fell apart after one wash . Threads c...,"bad thing, string, set, thread, product info, ...",It fell apart after one wash One set fell apar...,it fell apart after one wash one set fell apar...,1,"tie, product info, bad thing, string, set","application, wash, negative; ; other, thread, ..."
3,3,"Comfortable, nice color, but LOTS of pilling. ...","Comfortable, nice color, but lots of pilling ...","purchase, good care, nice color, fit, fabric, ...","nice color, I like the fit, weight, I would ha...","nice color, i like the fit, weight, i would ha...",3,"weight, thats, good care","service, good care, positive; emotion, good ca..."
4,4,"Cute dress that.even a short, skinny old woman...","Cute dress that.even a short, skinny old woma...","necessary spot, material, old woman, woman ski...","Cute dress that.even a short, skinny old woman...","cute dress that.even a short, skinny old woman...",5,"dress that.even, old woman, woman skinny, nece...","; emotion, cute dress, positive; properties, m..."


In [15]:
df_int

,reviewer_id,key_score_aspect
0,0,"type, name, positive; other, compliment, posit..."
1,1,"; properties, material, positive; application,..."
2,2,"application, wash, negative; ; other, thread, ..."
3,3,"service, good care, positive; emotion, good ca..."
4,4,"; emotion, cute dress, positive; properties, m..."
...,...,...
95,95,"type, new dress, positive; service, product, n..."
96,96,"application, color wash, negative; type, dress..."
97,97,"type, dress great/great dress, positive; emoti..."
98,98,"; emotion, poor quality, negative; application..."


In [16]:
result.to_csv("./clothes_result/final_nodes.csv", index=False)
df_base.to_csv("./clothes_result/base_clothes_v2.csv", index=False)